In [264]:
background = '0'
NORTH = 0
EAST = 1
SOUTH = 2
WEST = 3
class cellElement():

    def setPosition(self, x, y):
        return
    def setOrientation(self, a):
        return
    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return
    def nextCell(self,entdir):
        return
    def setNextCWRotation(self):
        return   
    def getView():
        return


In [265]:
class GameGrid():
    gx = 0

    def __init__ (self, height, width):
        self.height = height
        self.width = width
        self.grid = []
        self.view = []
        for i in range(0, width):
            self.grid.append([])
            self.view.append([])
            for j in range(0, height):
                c = regularRoad(True, self.grid)
                c.setPosition(i,j)
                self.grid[i].append(c)
                self.view[i].append(c.visuals)


    def addElement(self, cellElm, x, y):
        cellElm.setPosition(x,y)
        self.grid[x][y] = cellElm
        self.view[x][y] = cellElm.visuals
        return

    def removeElement(self, x, y):
        empty = regularRoad(True, self.grid)
        self.grid[x][y] = empty
        self.view[x][y] = '_' #display for BG
        return

    def display(self):
        for i in range(0,self.width):
            for j in range(0, self.height):
                print(self.view[i][j], end=' ')
            print('\n')

    def updateView(self):
        return

    def startSimulation(self): 
        return

    def setPauseResume(self):
        return

    def stopSimulation(self):
        return


In [266]:

class regularRoad(cellElement):
    def __init__(self, isStraight, gridRef):
        self.visuals = 'R'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.x = -1
        self.y = -1
        if(isStraight):
            self.dir1 = SOUTH
            self.dir2 = NORTH
        else: # default is a Right turn as in the pdf.
            # rotate this one to get a left turn if needed
            self.dir1 = SOUTH
            self.dir2 = EAST
        return

    def setPosition(self, x, y):
        self.x = x
        self.y = y
        return    
    def setCwRot(self):
        self.dir1 = (self.dir1 + 1) % 4
        self.dir2 = (self.dir2 + 1) % 4
        return

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return
    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir variable. and checking if next cell is available/connected.
        # print(self.myGrid.grid[self.x+1][self.y+1])
        return self.myGrid.grid[self.x+1][self.y+1]
        # if(entdir == NORTH):
        #     # exit south, y+1, x unchanged
        #     print(self.myGrid[self.y+1][self.x])
        #     return self.myGrid[self.y+1][self.x]

        # if(entdir == self.dir1):
        #     if(self.dir2 == NORTH):
                
        #     #north 0'sa X sabit, Y aaliyor
        
        # return
    def getPos(self):
        return self.x, self.y
    def getView(self):
        return  self.visuals


In [267]:
class switchRoad(cellElement):
    def __init__(self, typeofSwitch):
        # create 'pieces' of the switch.
        self.visuals = 'S'
        self.rotationCount = 0
        self.switchType = typeofSwitch
        self.pieces = {'direct' : regularRoad(True)}
        self.activePiece = self.pieces['direct']

        if(self.switchType == 1):
            self.pieces['rightTurn'] = regularRoad(False)
        
        elif(self.switchType == 2):
            self.pieces['leftTurn'] = regularRoad(False)
            self.pieces['leftTurn'].setOrientation(1, False)
       
        elif(self.switchType == 3): 
            self.pieces['rightTurn'] = regularRoad(False)
            self.pieces['leftTurn'] = regularRoad(False)
            self.pieces['leftTurn'].setOrientation(1, False)
        
        print(self.pieces)
        return

    def setPosition(self, x, y):
        self.x = x
        self.y = y
        return    
    def setCwRot(self): 
        # straightforward 90 degree rotation: S->W, W -> N and so on.
        if(self.switchType == 1):
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        elif(self.switchType == 2):
            self.pieces['leftTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        else: #switchType is 3
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
            self.pieces['leftTurn'].setOrientation(1)

        return

    def setOrientation(self, rotationAmount):
        # rotate 90 degrees CW, directly change dir variables.
        self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        # only for switch roads. change which piece is active.
        if(self.switchType == 1):
            # if right make direct, if direct make right
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 2):
            #if left make direct, if direct make left
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 3): 
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            elif(self.activePiece == self.pieces['rightTurn']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']
        return

    def getDuration(self, entdir):
        # It takes one second to pass this cell.
        return 1
    def getStop(self, entdir):
        return 5
    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir variable. and checking if next cell is available/connected.
        
        return
    def getView(self):
        return self.visuals

In [268]:
# class Tests():
#     def testRegularRoadOrientation(self, rotCount):
#         r=regularRoad(True)
        
#         print(r.dir1,r.dir2)
#         r.setOrientation(1)
#         print(r.dir1,r.dir2)


# orientTest = Tests()
# orientTest.testRegularRoadOrientation(1)


In [269]:
# class Tests():
#     def testSwitchRoadOrientation(self, rotCount):
        
#         s=switchRoad(2)
#         print(s.pieces['direct'].dir1, s.pieces['direct'].dir2,s.pieces['leftTurn'].dir1, s.pieces['leftTurn'].dir2)
#         s.setOrientation(1)
#         print(s.pieces['direct'].dir1, s.pieces['direct'].dir2,s.pieces['leftTurn'].dir1, s.pieces['leftTurn'].dir2)

#         # print(s.directRoad.dir1,s.directRoad.dir2)#,s.leftTurn.dir1, s.leftTurn.dir2)


# orientTest = Tests()
# orientTest.testSwitchRoadOrientation(1)


In [270]:
g = GameGrid(5,7) # width x height
c1 = regularRoad(True,g)
c2 = regularRoad(True,g)

cNext = regularRoad(True, g)
g.addElement(cNext, 1,2)
g.addElement(c1,0,1)
g.addElement(c2,2,2)
# g.addElement(regularRoad(True), 5, 4)
g.display()

nextRoad = c1.nextCell(NORTH)
print(nextRoad.getPos(), nextRoad.getView())

nextRoad = c2.nextCell(NORTH)
print(nextRoad.getPos(), nextRoad.getView())

nextRoad = cNext.nextCell(NORTH)
print(nextRoad.getPos(), nextRoad.getView())

R R R R R 

R R R R R 

R R R R R 

R R R R R 

R R R R R 

R R R R R 

R R R R R 

(1, 2) R
(3, 3) R
(2, 3) R
